# Chapter 06-04: 텍스트 분류 (Text Classification)

## 학습 목표
- IMDB 영화 리뷰 데이터셋으로 감성 분류(긍정/부정)를 구현한다.
- 4가지 모델 아키텍처(BoW, LSTM, GRU, BiLSTM)를 비교한다.
- `TextVectorization`을 활용한 엔드-투-엔드 파이프라인을 구성한다.
- 각 모델의 검증 정확도와 학습 시간을 비교한다.

In [ ]:
# 기본 라이브러리 임포트
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import time

# 한글 폰트 설정 (macOS)
matplotlib.rcParams['font.family'] = 'AppleGothic'
matplotlib.rcParams['axes.unicode_minus'] = False

print(f"TensorFlow 버전: {tf.__version__}")

# 재현성을 위한 시드 설정
tf.random.set_seed(42)
np.random.seed(42)

In [ ]:
# IMDB 데이터셋 로드 및 TextVectorization 적용

# 하이퍼파라미터 설정
MAX_TOKENS  = 10000  # 어휘 사전 최대 크기
MAX_SEQ_LEN = 200    # 시퀀스 최대 길이
EMBED_DIM   = 64     # 임베딩 차원
BATCH_SIZE  = 32     # 배치 크기
EPOCHS      = 5      # 학습 에포크 수

# IMDB 데이터셋 로드 (정수 인코딩된 버전)
# num_words: 빈도 순위 상위 MAX_TOKENS개 단어만 사용
print("IMDB 데이터셋 로드 중...")
(x_train_raw, y_train), (x_test_raw, y_test) = tf.keras.datasets.imdb.load_data(
    num_words=MAX_TOKENS
)
print(f"  훈련 샘플: {len(x_train_raw):,d}개")
print(f"  테스트 샘플: {len(x_test_raw):,d}개")
print(f"  레이블: 0=부정, 1=긍정")
print()

# IMDB 단어 인덱스 → 원문 복원
word_index = tf.keras.datasets.imdb.get_word_index()
# 인덱스 3부터 시작 (0=패딩, 1=시작, 2=미등록)
reverse_word_index = {v + 3: k for k, v in word_index.items()}
reverse_word_index[0] = '<PAD>'
reverse_word_index[1] = '<START>'
reverse_word_index[2] = '<UNK>'

def decode_review(encoded):
    """정수 시퀀스를 원문 텍스트로 복원"""
    return ' '.join(reverse_word_index.get(i, '?') for i in encoded)

print("샘플 리뷰 (처음 50단어):")
print(decode_review(x_train_raw[0][:50]))
print(f"레이블: {'긍정' if y_train[0] == 1 else '부정'}")
print()

# 패딩: 시퀀스 길이를 MAX_SEQ_LEN으로 통일
x_train = tf.keras.utils.pad_sequences(
    x_train_raw, maxlen=MAX_SEQ_LEN, padding='post', truncating='post'
)
x_test = tf.keras.utils.pad_sequences(
    x_test_raw, maxlen=MAX_SEQ_LEN, padding='post', truncating='post'
)
print(f"패딩 후 훈련 데이터 형태: {x_train.shape}")
print(f"패딩 후 테스트 데이터 형태: {x_test.shape}")

In [ ]:
# 모델 1: Embedding + GlobalAveragePooling (Bag of Words)
# 순서 정보를 무시하고 단어 임베딩의 평균을 사용하는 가장 단순한 방법

def build_bow_model():
    """Bag-of-Words 방식: 임베딩 평균 사용"""
    model = tf.keras.Sequential([
        # Embedding: 정수 인덱스 → 밀집 벡터
        tf.keras.layers.Embedding(MAX_TOKENS + 1, EMBED_DIM,
                                  mask_zero=True, name='embedding'),
        # GlobalAveragePooling1D: 시퀀스 차원을 평균으로 압축
        # (배치, 시퀀스, 임베딩) → (배치, 임베딩)
        tf.keras.layers.GlobalAveragePooling1D(name='global_avg_pool'),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dropout(0.3),
        tf.keras.layers.Dense(1, activation='sigmoid')  # 이진 분류
    ], name='BoW_Model')
    return model

model_bow = build_bow_model()
model_bow.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)
model_bow.summary()

In [ ]:
# 모델 2: Embedding + LSTM

def build_lstm_model():
    """단방향 LSTM 분류 모델"""
    model = tf.keras.Sequential([
        tf.keras.layers.Embedding(MAX_TOKENS + 1, EMBED_DIM,
                                  mask_zero=True, name='embedding'),
        # LSTM: 순서 정보를 유지하며 시퀀스 처리
        tf.keras.layers.LSTM(64, name='lstm'),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dropout(0.3),
        tf.keras.layers.Dense(1, activation='sigmoid')
    ], name='LSTM_Model')
    return model

model_lstm = build_lstm_model()
model_lstm.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)
model_lstm.summary()

In [ ]:
# 모델 3: Embedding + GRU

def build_gru_model():
    """GRU 분류 모델 - LSTM보다 경량"""
    model = tf.keras.Sequential([
        tf.keras.layers.Embedding(MAX_TOKENS + 1, EMBED_DIM,
                                  mask_zero=True, name='embedding'),
        # GRU: LSTM보다 파라미터가 적어 학습 속도 빠름
        tf.keras.layers.GRU(64, name='gru'),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dropout(0.3),
        tf.keras.layers.Dense(1, activation='sigmoid')
    ], name='GRU_Model')
    return model

model_gru = build_gru_model()
model_gru.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)
model_gru.summary()

In [ ]:
# 모델 4: Embedding + Bidirectional LSTM

def build_bilstm_model():
    """양방향 LSTM 분류 모델 - 양쪽 문맥 포착"""
    model = tf.keras.Sequential([
        tf.keras.layers.Embedding(MAX_TOKENS + 1, EMBED_DIM,
                                  mask_zero=True, name='embedding'),
        # Bidirectional: 순방향과 역방향 LSTM을 결합
        # 출력 차원: LSTM(64) × 2 = 128
        tf.keras.layers.Bidirectional(
            tf.keras.layers.LSTM(64),
            name='bilstm'
        ),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dropout(0.3),
        tf.keras.layers.Dense(1, activation='sigmoid')
    ], name='BiLSTM_Model')
    return model

model_bilstm = build_bilstm_model()
model_bilstm.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)
model_bilstm.summary()

In [ ]:
# 4개 모델 학습 및 검증 정확도 비교

results = {}  # 학습 결과 저장

models_to_train = [
    ('BoW (GlobalAvgPool)', model_bow),
    ('LSTM',               model_lstm),
    ('GRU',                model_gru),
    ('Bidirectional LSTM', model_bilstm),
]

for model_name, model in models_to_train:
    print(f"\n{'='*50}")
    print(f"학습 중: {model_name}")
    print('='*50)
    
    start_time = time.time()
    
    history = model.fit(
        x_train, y_train,
        epochs=EPOCHS,
        batch_size=BATCH_SIZE,
        validation_split=0.2,  # 훈련 데이터의 20%를 검증에 사용
        verbose=1
    )
    
    elapsed = time.time() - start_time
    
    # 테스트 평가
    test_loss, test_acc = model.evaluate(x_test, y_test, verbose=0)
    
    results[model_name] = {
        'history': history,
        'test_accuracy': test_acc,
        'test_loss': test_loss,
        'time': elapsed
    }
    
    print(f"테스트 정확도: {test_acc:.4f} | 학습 시간: {elapsed:.1f}초")

In [ ]:
# 4개 모델 성능 비교 시각화

model_names  = list(results.keys())
test_accs    = [results[m]['test_accuracy'] for m in model_names]
train_times  = [results[m]['time'] for m in model_names]

fig, axes = plt.subplots(1, 3, figsize=(16, 5))

# (1) 테스트 정확도 막대 차트
colors = ['#4C72B0', '#DD8452', '#55A868', '#C44E52']
bars = axes[0].bar(model_names, test_accs, color=colors, edgecolor='black', linewidth=0.8)
axes[0].set_ylim([0.8, 1.0])
axes[0].set_ylabel('테스트 정확도')
axes[0].set_title('모델별 테스트 정확도 비교')
axes[0].tick_params(axis='x', rotation=30)
for bar, acc in zip(bars, test_accs):
    axes[0].text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.002,
                 f'{acc:.4f}', ha='center', va='bottom', fontsize=10)

# (2) 학습 시간 막대 차트
bars2 = axes[1].bar(model_names, train_times, color=colors, edgecolor='black', linewidth=0.8)
axes[1].set_ylabel('학습 시간 (초)')
axes[1].set_title('모델별 학습 시간 비교')
axes[1].tick_params(axis='x', rotation=30)
for bar, t in zip(bars2, train_times):
    axes[1].text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.5,
                 f'{t:.1f}s', ha='center', va='bottom', fontsize=10)

# (3) 학습 곡선 비교
for i, (name, res) in enumerate(results.items()):
    val_acc = res['history'].history['val_accuracy']
    axes[2].plot(range(1, len(val_acc)+1), val_acc,
                 marker='o', label=name, color=colors[i])
axes[2].set_xlabel('에포크')
axes[2].set_ylabel('검증 정확도')
axes[2].set_title('에포크별 검증 정확도')
axes[2].legend(fontsize=8)
axes[2].grid(True, alpha=0.3)

plt.suptitle('IMDB 감성 분석: 4가지 모델 비교', fontsize=14)
plt.tight_layout()
plt.show()

# 최종 성능 요약 테이블
print("\n=== 최종 성능 요약 ===")
print(f"{'모델':<22} {'테스트 정확도':>14} {'학습 시간':>12}")
print('-' * 52)
for name in model_names:
    acc  = results[name]['test_accuracy']
    t    = results[name]['time']
    print(f"{name:<22} {acc:>14.4f} {t:>10.1f}초")

## 결과 분석 및 정리

### 실험 결과 해석

| 모델 | 특징 | 예상 정확도 | 예상 학습 시간 |
|------|------|-------------|----------------|
| **BoW (GlobalAvgPool)** | 순서 무시, 매우 빠름 | ~86% | 가장 빠름 |
| **LSTM** | 순서 포착, 장기 의존성 | ~87~88% | 중간 |
| **GRU** | LSTM과 유사, 경량 | ~87~88% | LSTM보다 빠름 |
| **Bidirectional LSTM** | 양방향 문맥 | ~88~89% | 가장 느림 |

### 주요 관찰
1. **BoW 모델**은 단순하지만 IMDB처럼 단어 빈도가 중요한 데이터에서도 준수한 성능을 보인다.
2. **LSTM/GRU**는 단어 순서("not good" vs "good not")를 구분할 수 있어 더 정확하다.
3. **BiLSTM**은 일반적으로 가장 높은 정확도를 달성하지만 학습 시간이 2배 이상이다.
4. 실무에서는 **정확도와 속도의 트레이드오프**를 고려하여 모델을 선택한다.

### 다음 챕터 예고
- **Chapter 06-05**: Seq2Seq 모델  
  인코더-디코더 구조로 입력 시퀀스를 다른 시퀀스로 변환하는 방법을 학습한다.